In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tqdm import tqdm_notebook

/home/ashutosh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Reading_Training_images

In [2]:
cat_img=[]
cat_labels=[]
dog_img=[]
dog_labels=[]


for i in range(1000):
    cat_img.append("/home/ashutosh/Documents/keras/data/train/cats/cat"+str(i)+".jpg")
    cat_labels.append([1,0])
    dog_img.append("/home/ashutosh/Documents/keras/data/train/dogs/dog."+str(i)+".jpg")
    dog_labels.append([0,1])
    
    
cat_dataset =tf.data.Dataset.from_tensor_slices((cat_img,cat_labels))

dog_dataset =tf.data.Dataset.from_tensor_slices((dog_img,dog_labels))

train_data=cat_dataset.concatenate(dog_dataset)

dataset=train_data.shuffle(buffer_size=2000)



# Reading_Validation_images

In [3]:
vcat_img=[]
vcat_labels=[]
vdog_img=[]
vdog_labels=[]


for i in range(400):
    vcat_img.append("/home/ashutosh/Documents/keras/data/validation/cats/cat"+str(i+1000)+".jpg")
    vcat_labels.append([1,0])
    vdog_img.append("/home/ashutosh/Documents/keras/data/validation/dogs/dog."+str(i+1000)+".jpg")
    vdog_labels.append([0,1])

vcat_dataset =tf.data.Dataset.from_tensor_slices((vcat_img,vcat_labels))
vdog_dataset =tf.data.Dataset.from_tensor_slices((vdog_img,vdog_labels))
vtrain_data=vcat_dataset.concatenate(vdog_dataset)
vdataset=vtrain_data.shuffle(buffer_size=800)
 

# Decoding,Augmenting,Resizing images

In [4]:
def _parse_function(filename, label):
    
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string,channels=3)
    image_resized = tf.image.resize_images(image_decoded, [30,30])
     
    return image_resized, label
def image_reshape(image, label):
    
    image = tf.reshape(image,[1,30,30,3])
    return image, label

def train_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

dataset = dataset.map(_parse_function)
#Storing_augmented_images
augment=dataset.map(train_preprocess)
dataset=dataset.concatenate(augment)
dataset = dataset.map(image_reshape)
dataset=dataset.shuffle(buffer_size=4000)
dataset = dataset.batch(40)


#Validation_dataset
vdataset = vdataset.map(_parse_function)
vdataset = vdataset.map(image_reshape)
vdataset = vdataset.batch(40)


# Reinitializable iterator to switch between data

In [5]:
iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)

features, labels = iterator.get_next()

# create the initialisation operations

train_init_op = iterator.make_initializer(dataset)

test_init_op = iterator.make_initializer(vdataset)




# Defining_convo_layer_&_Weights

In [6]:
def weight_variable(shape):
    # break simmetry
    w = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(w)

def bias_variable(shape):
    # avoid dead neurons
    b = tf.constant(0.1, shape=shape)
    return tf.Variable(b)

# pool
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# creates default conv layer
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling):
 
    shape = [filter_size, filter_size, num_input_channels, num_filters]
 
    weights = weight_variable(shape)
    biases = bias_variable([num_filters])
 
    
    layer = tf.nn.relu(tf.nn.conv2d(input=input,
                                    filter=weights,
                                    strides=[1, 1, 1, 1],
                                    padding='SAME') + biases)

    if use_pooling: 
        return max_pool_2x2(layer), weights

    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()

    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

def new_fc_layer(input, num_inputs, num_outputs, use_relu=True): 
    weights = weight_variable([num_inputs, num_outputs])
    biases = bias_variable([num_outputs])
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

# Vgg16_first_block

In [7]:
x = features
x_image = tf.reshape(x, [-1,30,30,3])

y_ = labels
# fist conv layer
convlayer1, w1 = new_conv_layer(x_image, 3, 3, 64,False)
# second conv layer
convlayer2, w2 = new_conv_layer(convlayer1, 64, 3, 64,True)
# flat layer
flat_layer, num_features = flatten_layer(convlayer2)
# fully connected layer
fclayer = new_fc_layer(flat_layer, num_features,40)
# DROPOUT

drop_layer = tf.nn.dropout(fclayer, keep_prob=0.5)
# final layer
W_f = weight_variable([40,2])
b_f = bias_variable([2])
y_f = tf.matmul(drop_layer, W_f) + b_f
y_f_softmax = tf.nn.softmax(y_f)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_f))

# train step
train_step = tf.train.AdamOptimizer(1e-6).minimize(loss)

# accuracy
correct_prediction = tf.equal(tf.argmax(y_f_softmax, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer())
    
    print("epochs,Training_accuracy:")
    for epoch in range(40):
        

        
        sess.run(train_init_op)
        tra_acc=0 
        loss_avg=0
        for i in tqdm_notebook(range(100)):
            
            
            train_s,training_acc,loss1=sess.run([train_step,accuracy,loss])          
            tra_acc=tra_acc+training_acc
            
            loss_avg=loss_avg+loss1
        avg_acc=float(tra_acc)/100.00  
        loss_avg=float(loss_avg)/100.00
        #Checkpoint_after_each_epoch        
        saver.save(sess, "/home/ashutosh/Documents/keras/data/"+str(epoch)+".ckpt")   
        print(epoch,avg_acc,loss_avg)
    
    
    
    Test_accuracy=0.0
     
    sess.run(test_init_op)
    for i in range(20):
        
        acc=sess.run(accuracy)
        Test_accuracy=Test_accuracy+acc
    print 'avg test accuracy in %:',100*Test_accuracy/20.0

epochs,Training_accuracy:


HBox(children=(IntProgress(value=0), HTML(value=u'')))


(0, 0.5004999962449074, 104.28122882843017)


HBox(children=(IntProgress(value=0), HTML(value=u'')))


(1, 0.49049999594688415, 101.62681205749512)


...


(14, 0.5024999979138375, 49.841959934234616)


HBox(children=(IntProgress(value=0), HTML(value=u'')))


(15, 0.5057499980926514, 48.65962869644165)


HBox(children=(IntProgress(value=0), HTML(value=u'')))

KeyboardInterrupt: 